# A Review on Adam Stochastic Optimization

Original Code: https://github.com/abe-shek/ML-model-comparison

In [1]:
# datasets 
from sklearn.datasets import load_digits 

# data processing
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import train_test_split  
from sklearn.metrics import accuracy_score 

# implemtation
import numpy as np
import numpy.random as r 

# plotting
import matplotlib.pyplot as plt 


## MNIST Loading

In [2]:
def convert_y_to_vect(y):
    y_vect = np.zeros((len(y), 10))
    for i in range(len(y)):
        y_vect[i, y[i]] = 1
    return y_vect

def load_mnist_data(source="sk"):
    if source == "sk":
        digits=load_digits()
        X = digits.data
        y = digits.target
        
    X_scale = StandardScaler()
    X = X_scale.fit_transform(X)
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    
    y_train = convert_y_to_vect(y_train)
#     y_test = convert_y_to_vect(y_test)
    
    return (x_train, y_train), (x_test, y_test)

## Neural Network Model

In [3]:
def initializers(nn_structure, acti="relu", mode="weights"):
    W = {}; b = {};
    dW = {}; db = {};
    VdW = {}; Vdb = {};
    SdW = {}; Sdb = {}
    for l in range(1, len(nn_structure)):
        if mode=="weights":
            [low, high, inter] = [0., 1., 0.]
            if acti=="sigmoid":
                inter = np.sqrt((6)/(nn_structure[l] + nn_structure[l-1]))
            elif acti=="relu":
                inter = np.sqrt((6.)/float(nn_structure[l] + nn_structure[l-1]))

            W[l] = r.uniform(low=-inter, high=inter, size=(nn_structure[l], nn_structure[l-1])) 
            b[l] = r.uniform(low=-inter, high=inter, size=(nn_structure[l],))
        elif mode=="gradients":
            dW[l] = np.zeros((nn_structure[l], nn_structure[l-1]))
            db[l] = np.zeros((nn_structure[l],))
        elif mode=="momentum":
            VdW[l] = np.zeros((nn_structure[l], nn_structure[l-1]))
            Vdb[l] = np.zeros((nn_structure[l],))
        elif mode=="rms_prop": 
            SdW[l] = np.zeros((nn_structure[l], nn_structure[l-1]))
            Sdb[l] = np.zeros((nn_structure[l],))
        elif mode=="adam": 
            VdW[l] = np.zeros((nn_structure[l], nn_structure[l-1]))
            Vdb[l] = np.zeros((nn_structure[l],))
            SdW[l] = np.zeros((nn_structure[l], nn_structure[l-1]))
            Sdb[l] = np.zeros((nn_structure[l],))
    if mode=="weights":
        return W, b
    elif mode=="gradients":
        return dW, db
    elif mode=="momentum":
        return VdW, Vdb
    elif mode=="rms_prop":
        return SdW, Sdb
    elif mode=="adam":
        return VdW, Vdb, SdW, Sdb
    return None, None


def regularizers(W, lamda, mode="l2"):
    if mode == "l2":
        l2_cost = 0.
        for l in range(1, len(W)):
            l2_cost += np.sum(np.square(W[l]))
        return (lamda/2.)*l2_cost
    return 0


def f(z, activation="relu", deri=False):
    if deri:
        fz_d = 0
        if activation == "sigmoid":
            fz_d = f(z, activation="sigmoid") * (1 - f(z, activation="sigmoid"))
        elif activation == "relu":
            fz_d = (z>=0).astype("int")
        return fz_d
    else:
        fz = 0
        if activation == "sigmoid":
            fz = 1 / (1 + np.exp(-z))
        elif activation == "relu":
            fz = np.maximum(0, z)
        return fz


def forward_prop(x, W, b, acti="relu"):
    a = {1: x}
    z = {} 
    for l in range(1, len(W) + 1):
        node_in = a[l]
        z[l+1] = W[l].dot(node_in) + b[l]  
        a[l+1] = f(z[l+1], activation=acti) 
    return a, z


def back_prop(y, a, z, W, b, dW, db, acti="relu", regularizer=[0.01, "l2"]):
    delta = {}
    cost = 0
    for l in range(len(a), 0, -1):
        if l == len(a):
            delta[l] = -(y-a[l]) * f(z[l], activation=acti, deri=True) 
            cost = np.linalg.norm((y-a[l]))
            if regularizer and len(regularizer) == 2:
                 cost += regularizers(W,regularizer[0],mode=regularizer[1])
        else:
            if l > 1:
                delta[l] = np.dot(np.transpose(W[l]), delta[l+1]) * f(z[l], activation=acti, deri=True)
                
            dW[l] += np.dot(delta[l+1][:,np.newaxis], np.transpose(a[l][:,np.newaxis]))
            db[l] += delta[l+1]
    return dW, db, cost


def plot_cost(cost):
    plt.plot(cost)
    plt.ylabel('Average J')
    plt.xlabel('Epochs')
    plt.show()

In [4]:
def train(nn_structure, X, y, params, verbose=0):
    [ext, activation, batch_size, epochs, alpha, reg] = params
    cnt = 1
    N = len(y)
    if not batch_size:
        batch_size = N
    avg_cost_func = []
    if verbose==1:
        print('Starting gradient descent for {} iterations\n\n'.format(epochs))    
    while cnt <= epochs:
        if verbose==1:
            print('Epoch {} of {}\n'.format(cnt, epochs))
        
        if cnt == 1:
             W, b = initializers(nn_structure, acti=activation, mode="weights")
        epoch_cost = 0
        for j in range(0, N, batch_size):
            avg_cost = 0        
            dW, db = initializers(nn_structure, acti=activation, mode="gradients")
            if ext and ext["name"] == "momentum":
                VdW, Vdb = initializers(nn_structure, mode="momentum")
            if ext and ext["name"] == "rms_prop":
                SdW, Sdb = initializers(nn_structure, mode="rms_prop")
            if ext and ext["name"] == "adam":
                VdW, Vdb, SdW, Sdb = initializers(nn_structure, mode="adam")

            for i in range(j, min(j+batch_size,N), 1):
                a, z = forward_prop(X[i, :], W, b, acti=activation)
                dW, db, cost = back_prop(y[i,:], a, z, W, b, dW, db, acti=activation, regularizer=reg)
                avg_cost += cost
                if ext and ext["name"] == "momentum":
                    VdW, Vdb = add_momentum(VdW, dW, Vdb, db, ext["params"]["beta"])
                if ext and ext["name"] == "rms_prop":
                    SdW, Sdb = rms_prop(SdW, dW, Sdb, db, ext["params"]["beta2"])
                if ext and ext["name"] == "adam":
                    VdW, Vdb, SdW, Sdb = adam(VdW, Vdb, SdW, dW, Sdb, db, ext["params"]["beta1"], ext["params"]["beta2"])

            for l in range(len(nn_structure) - 1, 0, -1):
                if ext and ext["name"] == "momentum":
                    W[l] += -alpha * (1.0/batch_size * VdW[l] + reg[0]*W[l])
                    b[l] += -alpha * (1.0/batch_size * Vdb[l])
                if ext and ext["name"] == "rms_prop":
                    W[l] += -alpha * (1.0/batch_size * (dW[l]/(np.sqrt(SdW[l]) + ext["params"]["epsilon"])) + reg[0]*W[l])
                    b[l] += -alpha * (1.0/batch_size * (db[l]/(np.sqrt(Sdb[l]) + ext["params"]["epsilon"])))
                if ext and ext["name"] == "adam":
                    # bias correction
                    t = j % batch_size + 1
                    VdW_corr = VdW[l]/(1-ext["params"]["beta1"]**t)
                    Vdb_corr = Vdb[l]/(1-ext["params"]["beta1"]**t)
                    SdW_corr = SdW[l]/(1-ext["params"]["beta2"]**t)
                    Sdb_corr = Sdb[l]/(1-ext["params"]["beta2"]**t)
                    
                    W[l] += -alpha * (1.0/batch_size * (VdW_corr/(np.sqrt(SdW_corr) + ext["params"]["epsilon"])) + reg[0]*W[l])
                    b[l] += -alpha * (1.0/batch_size * (Vdb_corr/(np.sqrt(Sdb_corr) + ext["params"]["epsilon"])))
                else:
                    W[l] += -alpha * (1.0/batch_size * dW[l] + reg[0]*W[l])
                    b[l] += -alpha * (1.0/batch_size * db[l])

            avg_cost = 1.0/batch_size * avg_cost
            epoch_cost += avg_cost
            
        avg_cost_func.append(epoch_cost/float(N/batch_size))
        cnt += 1
    if verbose>=1:
        plot_cost(avg_cost_func)
    return W, b, avg_cost_func


def evaluate(X, y, W, b, n_layers, verbose=0):
    N = X.shape[0]
    y_pred = np.zeros((N,))
    for i in range(N):
        a, z = forward_prop(X[i, :], W, b)
        y_pred[i] = np.argmax(a[n_layers])
    accuracy = accuracy_score(y, y_pred)
    print('Prediction accuracy = {:0.2f} %'.format(accuracy * 100))
    return accuracy
    
    
def compile_nn(source="sk", ext=None, lr=0.1, num_runs=1, batch_size=None, epochs=300, verbose=0):
    nn_structure = [64 if source=="sk" else 784, 30, 10]
    params = [ext, "relu", batch_size, epochs, lr, [0.01,"l2"]]
    [_, activation,_, epochs, alpha, reg] = params
    
    if verbose>=1:
        print("\n---------------------------\n")
        print("Training Nerual Network with - \n\nStructure - {}\nExtension - {}\nActivation Function - {}\nBatch Size - {}\nEpochs - {}\nLearning Rate - {}\nRegularization - {}".format(nn_structure, ext if ext else "Naive", activation, batch_size if batch_size else "Full", epochs, alpha, reg))
        print("\n---------------------------\n")
    elif ext:
        print("Training with extension - {}".format(ext))
    
    avg_accuracy = []
    
    for i in range(1, num_runs+1):
        (X_train, y_train), (X_test, y_test) = load_mnist_data(source)
        W, b, avg_cost_func = train(nn_structure, X_train, y_train, params, verbose)
        avg_accuracy.append(evaluate(X_test, y_test, W, b, 3, verbose))
    
    if num_runs > 1:
        print("Average accuracy over {} runs = {:0.2f} %".format(num_runs, (np.sum(np.array(avg_accuracy))/float(num_runs)*100)))

## SGD Algorithms

In [5]:
def add_momentum(VdW, dW, Vdb, db, beta=0.9):
    for l in range(1, len(dW)+1):
        VdW[l] = beta*VdW[l] + (1.-beta)*dW[l]
        Vdb[l] = beta*Vdb[l] + (1.-beta)*db[l]
    return VdW, Vdb

In [6]:
def rms_prop(SdW, dW, Sdb, db, beta2=0.999):
    for l in range(1, len(dW)+1):
        SdW[l] = beta2*SdW[l] + (1.-beta2)*(dW[l]**2)
        Sdb[l] = beta2*Sdb[l] + (1.-beta2)*(db[l]**2)
    return SdW, Sdb

In [7]:
def adam(VdW, Vdb, SdW, dW, Sdb, db, beta1=0.9, beta2=0.999):
    for l in range(1, len(dW)+1):
        VdW[l] = beta1*VdW[l] + (1.-beta1)*dW[l]
        Vdb[l] = beta1*Vdb[l] + (1.-beta1)*db[l]
        SdW[l] = beta2*SdW[l] + (1.-beta2)*(dW[l]**2)
        Sdb[l] = beta2*Sdb[l] + (1.-beta2)*(db[l]**2)
    return VdW, Vdb, SdW, Sdb

## Training (5 epochs)

In [8]:
compile_nn(ext=None, num_runs=5, batch_size=32, epochs=5)

Prediction accuracy = 55.49 %
Prediction accuracy = 66.76 %
Prediction accuracy = 69.54 %
Prediction accuracy = 69.40 %
Prediction accuracy = 83.45 %
Average accuracy over 5 runs = 68.93 %


In [19]:
ext={"name":"momentum", "params":{"beta":0.9}}
compile_nn(ext=ext, lr=0.1, num_runs=5, batch_size=32, epochs=5)

Training with extension - {'name': 'momentum', 'params': {'beta': 0.9}}
Prediction accuracy = 52.16 %
Prediction accuracy = 55.22 %
Prediction accuracy = 51.32 %
Prediction accuracy = 57.72 %
Prediction accuracy = 64.39 %
Average accuracy over 5 runs = 56.16 %


In [10]:
ext={"name":"rms_prop", "params":{"beta2":0.9, "epsilon":1e-7}}
compile_nn(ext=ext, lr=0.01, num_runs=5, batch_size=32, epochs=5)

Training with extension - {'name': 'rms_prop', 'params': {'beta2': 0.9, 'epsilon': 1e-07}}
Prediction accuracy = 37.69 %
Prediction accuracy = 53.41 %
Prediction accuracy = 54.24 %
Prediction accuracy = 52.43 %
Prediction accuracy = 56.75 %
Average accuracy over 5 runs = 50.90 %


In [20]:
ext={"name":"adam", "params":{"beta1":0.9, "beta2":0.999, "epsilon":1e-8}}
compile_nn( ext=ext, lr=0.1, num_runs=5, batch_size=32, epochs=5)

Training with extension - {'name': 'adam', 'params': {'beta1': 0.9, 'beta2': 0.999, 'epsilon': 1e-08}}
Prediction accuracy = 60.22 %
Prediction accuracy = 78.30 %
Prediction accuracy = 81.22 %
Prediction accuracy = 85.54 %
Prediction accuracy = 45.20 %
Average accuracy over 5 runs = 70.10 %


## Training (300 epochs)

In [12]:
compile_nn(ext=None, num_runs=5, batch_size=32, epochs=300)

Prediction accuracy = 98.19 %
Prediction accuracy = 79.00 %
Prediction accuracy = 87.34 %
Prediction accuracy = 87.34 %
Prediction accuracy = 90.68 %
Average accuracy over 5 runs = 88.51 %


In [13]:
ext={"name":"momentum", "params":{"beta":0.9}}
compile_nn(ext=ext, lr=0.1, num_runs=5, batch_size=32, epochs=300)

Training with extension - {'name': 'momentum', 'params': {'beta': 0.9}}
Prediction accuracy = 69.26 %
Prediction accuracy = 64.67 %
Prediction accuracy = 98.05 %
Prediction accuracy = 88.32 %
Prediction accuracy = 87.62 %
Average accuracy over 5 runs = 81.59 %


In [14]:
ext={"name":"rms_prop", "params":{"beta2":0.9, "epsilon":1e-7}}
compile_nn(ext=ext, lr=0.01, num_runs=5, batch_size=32, epochs=300)

Training with extension - {'name': 'rms_prop', 'params': {'beta2': 0.9, 'epsilon': 1e-07}}
Prediction accuracy = 79.69 %
Prediction accuracy = 95.41 %
Prediction accuracy = 78.86 %
Prediction accuracy = 95.69 %
Prediction accuracy = 79.14 %
Average accuracy over 5 runs = 85.76 %


In [15]:
ext={"name":"adam", "params":{"beta1":0.9, "beta2":0.999, "epsilon":1e-8}}
compile_nn( ext=ext, lr=0.1, num_runs=5, batch_size=32, epochs=300)

Training with extension - {'name': 'adam', 'params': {'beta1': 0.9, 'beta2': 0.999, 'epsilon': 1e-08}}
Prediction accuracy = 76.36 %
Prediction accuracy = 96.38 %
Prediction accuracy = 85.26 %
Prediction accuracy = 94.99 %
Prediction accuracy = 79.00 %
Average accuracy over 5 runs = 86.40 %
